# Analisis individual de variables

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from scipy.stats import pointbiserialr, chi2_contingency, f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import LabelEncoder

#### Esta celda realiza dos funciones esenciales: importar las librerías necesarias para el análisis estadístico y cargar el conjunto de datos preprocesado en el entorno de Python.

In [18]:
import pandas as pd
import numpy as np
from scipy.stats import pointbiserialr

try:
    df = pd.read_csv('data_cleaner1.csv')
    print("DataFrame cargado exitosamente. Primeras 5 filas:")
    print(df.head())
except FileNotFoundError:
    print("Error: Asegúrate de cargar el archivo CSV preprocesado en tu entorno de Colab.")

print("\nColumnas del DataFrame:")
print(df.columns.tolist())

DataFrame cargado exitosamente. Primeras 5 filas:
   AgeMonths  WeightKg  Vaccinated  HealthCondition  TimeInShelterDays  \
0   0.742691 -1.281098           1                0          -0.659602   
1  -0.369797  0.045756           0                0          -1.397929   
2   0.838595 -1.637043           0                0           1.594240   
3   0.090543 -1.485328           0                0           0.661616   
4   0.589244  0.575608           0                0          -0.620743   

   AdoptionFee  PreviousOwner  AdoptionLikelihood  PetType_Bird  PetType_Cat  \
0    -0.764025              0                   0           1.0          0.0   
1    -0.098998              0                   0           0.0          0.0   
2     0.951044              0                   0           0.0          0.0   
3    -0.225003              1                   0           1.0          0.0   
4    -1.646060              1                   0           0.0          0.0   

   PetType_Dog  Breed_Go

#### Esta celda define las listas de las variables predictoras (predictoras) y la variable objetivo (target). Esto es crucial para asegurar que solo las características que se someterán al análisis de discriminación (excluyendo la variable de resultado) sean procesadas en el siguiente paso.

In [19]:
# Lista de todas las variables predictoras a analizar
predictoras = [
    'AgeMonths', 'WeightKg', 'Vaccinated', 'HealthCondition',
    'TimeInShelterDays', 'AdoptionFee', 'PreviousOwner',
    'PetType_Bird', 'PetType_Cat', 'PetType_Dog',
    'Breed_Golden Retriever', 'Breed_Labrador', 'Breed_Poodle', 'Breed_Siamese',
    'Color_Black', 'Color_Brown', 'Color_Gray', 'Color_Orange',
    'Size_Encoded'
]

target = 'AdoptionLikelihood'

Esta es la fase central del análisis individual. Utilizamos la función Correlación Punto-Biserial (pointbiserialr) para medir la fuerza de la asociación lineal entre cada característica y la variable objetivo binaria (AdoptionLikelihood)

$r_{pb}$ (Punto-Biserial): Mide la correlación entre una variable continua (como AgeMonths) y una variable binaria (como AdoptionLikelihood).$r_{\Phi}$ (Phi): Mide la correlación entre dos variables binarias (como Vaccinated y AdoptionLikelihood).

Meta: Cuantificar la capacidad de cada característica para distinguir entre las mascotas adoptadas y las no adoptadas. El resultado se organiza y se muestra por la fuerza absoluta de la correlación.

In [20]:
resultados_discriminacion = {}

print("Iniciando análisis de Correlación Punto-Biserial (r_pb)...")

for col in predictoras:
    # pointbiserialr retorna una tupla: (coeficiente_r, p_valor)
    try:
        r_val, p_val = pointbiserialr(df[col], df[target])

        # Almacenar el resultado
        resultados_discriminacion[col] = {
            'r_pb_phi': r_val,
            'p_value': p_val,
            '|r_pb_phi|': abs(r_val)
        }
    except Exception as e:
        # Manejo de errores, por si alguna columna tiene valores no numéricos
        print(f"Error al procesar la columna {col}: {e}")

# Convertir los resultados a un DataFrame para fácil lectura y ordenar por fuerza de correlación
df_analisis = pd.DataFrame(resultados_discriminacion).T.sort_values(by='|r_pb_phi|', ascending=False)

print("\n--- RESULTADOS DE LA CAPACIDAD DISCRIMINATIVA ---")
print(df_analisis)

Iniciando análisis de Correlación Punto-Biserial (r_pb)...

--- RESULTADOS DE LA CAPACIDAD DISCRIMINATIVA ---
                        r_pb_phi       p_value  |r_pb_phi|
Vaccinated              0.301311  2.144218e-43    0.301311
Breed_Labrador          0.272174  2.012860e-35    0.272174
HealthCondition        -0.244061  1.323262e-28    0.244061
AgeMonths              -0.202209  5.806012e-20    0.202209
PetType_Dog             0.170754  1.344010e-14    0.170754
PetType_Cat            -0.050898  2.259215e-02    0.050898
PetType_Bird           -0.031944  1.525557e-01    0.031944
Breed_Golden Retriever -0.024121  2.801031e-01    0.024121
PreviousOwner          -0.023065  3.016928e-01    0.023065
Color_Orange            0.020293  3.635270e-01    0.020293
WeightKg                0.017601  4.306552e-01    0.017601
Breed_Siamese          -0.016215  4.678155e-01    0.016215
Color_Gray              0.015032  5.009207e-01    0.015032
Color_Black             0.014147  5.264538e-01    0.014147
Size_

Esta celda aplica el filtro de selección de características basado en los resultados de la correlación.Criterio: Se eliminan las características cuya fuerza de correlación absoluta con el target sea muy baja (ej. $|r| < 0.10$) Y cuya relación sea estadísticamente insignificante ($p > 0.05$).

In [21]:

UMBRAL_CORRELACION = 0.10
UMBRAL_P_VALOR = 0.05

candidatas_eliminacion = df_analisis[
    (df_analisis['|r_pb_phi|'] < UMBRAL_CORRELACION) &
    (df_analisis['p_value'] > UMBRAL_P_VALOR)
]

# Imprimir el informe final
print("\n--- CARACTERÍSTICAS CANDIDATAS A SER ELIMINADAS (Baja Discriminación) ---")
if not candidatas_eliminacion.empty:
    print(f"Criterio: |r| < {UMBRAL_CORRELACION} Y p > {UMBRAL_P_VALOR}")
    print(candidatas_eliminacion[['r_pb_phi', 'p_value']])
else:
    print("No se encontraron candidatas a ser eliminadas bajo los criterios definidos.")

# Lista de variables a conservar para el siguiente paso (Análisis de Redundancia)
conservar = df_analisis[
    (df_analisis['|r_pb_phi|'] >= UMBRAL_CORRELACION) |
    (df_analisis['p_value'] <= UMBRAL_P_VALOR)
].index.tolist()

print(f"\nVariables a conservar para el análisis de redundancia: {conservar}")


--- CARACTERÍSTICAS CANDIDATAS A SER ELIMINADAS (Baja Discriminación) ---
Criterio: |r| < 0.1 Y p > 0.05
                        r_pb_phi   p_value
PetType_Bird           -0.031944  0.152556
Breed_Golden Retriever -0.024121  0.280103
PreviousOwner          -0.023065  0.301693
Color_Orange            0.020293  0.363527
WeightKg                0.017601  0.430655
Breed_Siamese          -0.016215  0.467816
Color_Gray              0.015032  0.500921
Color_Black             0.014147  0.526454
Size_Encoded           -0.012117  0.587472
TimeInShelterDays       0.008867  0.691356
AdoptionFee            -0.005486  0.805990
Breed_Poodle            0.004477  0.841138
Color_Brown            -0.002048  0.926958

Variables a conservar para el análisis de redundancia: ['Vaccinated', 'Breed_Labrador', 'HealthCondition', 'AgeMonths', 'PetType_Dog', 'PetType_Cat']


Esta celda calcula la Matriz de Correlación de Pearson entre las variables predictoras que ya demostraron una alta capacidad discriminativa individual (las variables_conservadas).

Criterio de Redundancia: Se busca identificar pares de variables con una correlación muy alta (típicamente $|r| > 0.70$).

Meta: Evitar la multicolinealidad, que ocurre cuando dos variables miden esencialmente la misma información. Si se encuentra redundancia, se debe eliminar una de las variables del par para simplificar y estabilizar el modelo final.

In [22]:
# Lista de variables predictoras que demostraron buena capacidad discriminativa
variables_conservadas = [
    'Vaccinated', 'Breed_Labrador', 'HealthCondition',
    'AgeMonths', 'PetType_Dog', 'PetType_Cat'
]

# Calcular la Matriz de Correlación de Pearson
matriz_correlacion = df[variables_conservadas].corr()

print("\n--- MATRIZ DE CORRELACIÓN ENTRE VARIABLES CONSERVADAS ---")
print("Busque valores absolutos |r| > 0.7 para identificar redundancia.")
print(matriz_correlacion.round(3))


--- MATRIZ DE CORRELACIÓN ENTRE VARIABLES CONSERVADAS ---
Busque valores absolutos |r| > 0.7 para identificar redundancia.
                 Vaccinated  Breed_Labrador  HealthCondition  AgeMonths  \
Vaccinated            1.000           0.025            0.024     -0.002   
Breed_Labrador        0.025           1.000           -0.021      0.000   
HealthCondition       0.024          -0.021            1.000      0.017   
AgeMonths            -0.002           0.000            0.017      1.000   
PetType_Dog           0.020           0.550           -0.007      0.008   
PetType_Cat          -0.005          -0.189            0.034     -0.010   

                 PetType_Dog  PetType_Cat  
Vaccinated             0.020       -0.005  
Breed_Labrador         0.550       -0.189  
HealthCondition       -0.007        0.034  
AgeMonths              0.008       -0.010  
PetType_Dog            1.000       -0.344  
PetType_Cat           -0.344        1.000  


### Características Candidatas a Ser Eliminadas

$$
\begin{array}{|l|c|c|l|}
    \hline
    \textbf{Característica} & \textbf{Métrica} & \textbf{Valor Clave} & \textbf{Justificación (Capacidad Discriminativa Baja)} \\
    \hline
    \text{AdoptionFee} & r_{pb} & -0.005 & \text{Correlación insignificante y muy cercana a cero.} \\
    \text{TimeInShelterDays} & r_{pb} & 0.008 & \text{Correlación insignificante y muy cercana a cero.} \\
    \text{WeightKg} & r_{pb} & 0.017 & \text{Correlación insignificante y muy cercana a cero.} \\
    \text{PreviousOwner} & r_{pb} & -0.023 & \text{Correlación insignificante y muy cercana a cero.} \\
    \text{Size\_Encoded} & r_{pb} & -0.012 & \text{Correlación insignificante y muy cercana a cero.} \\
    \text{PetType\_Bird} & r_{pb} & -0.031 & \text{Baja correlación e insignificante.} \\
    \text{Breed\_Poodle} & r_{\Phi} & 0.004 & \text{Correlación insignificante y muy cercana a cero.} \\
    \text{Breed\_Golden Retriever} & r_{\Phi} & -0.024 & \text{Correlación insignificante y muy cercana a cero.} \\
    \text{Breed\_Siamese} & r_{\Phi} & -0.016 & \text{Correlación insignificante y muy cercana a cero.} \\
    \text{Color\_Black} & r_{\Phi} & 0.014 & \text{Correlación insignificante y muy cercana a cero.} \\
    \text{Color\_Brown} & r_{\Phi} & -0.002 & \text{Correlación insignificante y muy cercana a cero.} \\
    \text{Color\_Gray} & r_{\Phi} & 0.015 & \text{Correlación insignificante y muy cercana a cero.} \\
    \text{Color\_Orange} & r_{\Phi} & 0.020 & \text{Correlación insignificante y muy cercana a cero.} \\
    \hline
\end{array}
$$